# Perte_Charge_Singuliere test case


## Introduction
 
 Validation made by: ll258196

 Report generated 11/07/2023

### Description
 
Test cases for using the Perte_charge_singuliere option to model a pressure drop across a surface (e.g. through a grid).


In [ ]:
from trustutils import run

run.TRUST_parameters("1.9.3")

run.initBuildDirectory()

In [ ]:
### Generate MED mesh #####

run.useMEDCoupling()
import medcoupling as mc
import numpy as np
import os

bld = run.BUILD_DIRECTORY
filename = os.path.join(bld,"Groupe_Faces_VDF","mesh.py")

mesh_file = os.path.join(bld,"Groupe_Faces_VDF","mesh.med")
partition_file = os.path.join(bld,"Groupe_Faces_VDF","Partition.med")

xarr = mc.DataArrayDouble(11,1)
xarr.iota(0.)
cmesh = mc.MEDCouplingCMesh()
cmesh.setCoords(xarr,xarr,xarr)
mesh = cmesh.buildUnstructured()
mesh.setName("mesh")

# define faces
mesh_fac, _, _, r, rI = mesh.buildDescendingConnectivity()
dsi = rI.deltaShiftIndex()
skin_i = dsi.findIdsEqual(1)
skin_i.setName("frontieres")
bary_fac = mesh_fac.computeCellCenterOfMass()

eps = 1.0e-4

bary_fac_z = bary_fac[:,2]
inlet = bary_fac_z.findIdsInRange(-eps, +eps)
inlet.setName("Inlet")
outlet = bary_fac_z.findIdsInRange(10.-eps, 10.+eps)
outlet.setName("Outlet")

wall = skin_i.buildSubstraction(inlet.buildUnion(outlet))
wall.setName("Adiab")

surface = bary_fac_z.findIdsInRange(5.-eps, 5.+eps)
surface.setName("interf")

#Recording
mfu = mc.MEDFileUMesh()
mfu.setMeshAtLevel(0, mesh)
mfu.setMeshAtLevel(-1, mesh_fac)
mfu.setGroupsAtLevel(-1, [inlet, outlet,wall,surface])
mfu.write(mesh_file, 2) # 2 stands for write from scratch

# Add Partitionnement
meshRead = mc.ReadUMeshFromFile(mesh_file,"mesh",0)

arr2 = mc.DataArrayInt32(meshRead.getNumberOfCells())

arr2[0:500] = 0
arr2[250:500] = 1
arr2[500:750] = 2
arr2[750:1000] = 3

f = mc.MEDCouplingFieldInt32(mc.ON_CELLS, mc.ONE_TIME)
f.setArray(arr2)
f.setMesh(meshRead)
f.setName("Thread")

mc.WriteField(partition_file,f,True)

In [ ]:
run.addCase("VDF","Perte_Charge_Singuliere_VDF.data",nbProcs=1)
run.addCase("VEF","Perte_Charge_Singuliere_VEF.data",nbProcs=1)


case = run.addCase("Groupe_Faces_VDF","PCS_med_groupe_faces_VDF.data",nbProcs=4) 
case.partition()
run.addCase("Groupe_Faces_VEF","PCS_med_groupe_faces_VEF.data",nbProcs=1) 
run.printCases()
run.runCases()

## With Sous domaine interface

Imposition of pressure loss at $X = 1$ in VDF and VEF

In [ ]:
from trustutils import visit

fig=visit.Show("VDF/Perte_Charge_Singuliere_VDF.lata","Pseudocolor","PRESSION_PA_ELEM_dom",plotmesh=True,title="Pressure_PA",nY=2,nX=1)
fig.add("VEF/Perte_Charge_Singuliere_VEF.lata","Pseudocolor","PRESSION_PA_ELEM_dom",xIndice=0,yIndice=1)
fig.plot()

## Using internal_faces into med file

### With VDF discretization 

We test the implementation with internal face group.  
Partitioning create joints common to the internal face to test the correct parallelization using groups of internal faces that must be consider in two different processors. 


In [ ]:

fig=visit.Show("Groupe_Faces_VDF/PAR_PCS_med_groupe_faces_VDF.lata","Subset","blocks(dom)",iteration=-1,plotmesh=True,nY=2,nX=1)
fig.rotation3D([45,45,45])
fig.add("Groupe_Faces_VDF/PAR_PCS_med_groupe_faces_VDF.lata","Pseudocolor","PRESSION_PA_ELEM_dom",xIndice=0,yIndice=1)
fig.rotation3D([45,45,45])
fig.plot()


### With VEF discretization 

Pressure loss imposed on a specific interface marked by a group of internal faces in the med file (in red)

![](src/internal_faces_VEF/interf_PCS.png) 


### DataFile

In [ ]:
run.dumpDataset("Groupe_Faces_VEF/PCS_med_groupe_faces_VEF.data",["interf"])

In [ ]:
fig=visit.Show("Groupe_Faces_VEF/PCS_med_groupe_faces_VEF.lata", "Pseudocolor", "PRESSION_PA_ELEM_dom")
fig.rotation3D([0,65,55])
fig.plot()